In [1]:
!pwd

/mnt/d/NickWorks/OCR/jupyterNotebooks


In [2]:
from google.cloud import vision
from google.cloud.vision import types
import io
from google.protobuf.json_format import MessageToDict

In [13]:
import fitz
file="deathcertificate3"
extension=".pdf"
current_file="../Dataset/NewDataset/"+file+extension
write_file_directory="../Dataset/NewDatasetImageExtracted/"
write_file_path=write_file_directory+file+".png"
if extension.endswith('.pdf'):
    doc = fitz.open(current_file)
    for i in range(len(doc)):
        for img in doc.getPageImageList(i):
            xref = img[0]
            pix = fitz.Pixmap(doc, xref)
            #if pix.n < 5:       # this is GRAY or RGB
            if False:
                pix.writePNG(write_file_path)
            else:   # CMYK: convert to RGB first
                pix1 = fitz.Pixmap(fitz.csRGB, pix)
                pix1.writePNG(write_file_path)
                pix1 = None
            pix = None
else:
    im = Image.open(current_file)
    im.save(write_file_path)

In [14]:
import cv2
import numpy as np
img = cv2.imread(write_file_path)
write_file_path=write_file_directory+file+"new.png"
gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray=cv2.bitwise_not(gray)
#thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
coords = np.column_stack(np.where(thresh > 0))
angle = cv2.minAreaRect(coords)[-1]
if angle < -45:
    angle = -(90 + angle)
else:
    angle = -angle
(h, w) = img.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle, 1.0)
rotated = cv2.warpAffine(img, M, (w, h),flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
# show the output image
print("[INFO] angle: {:.3f}".format(angle))
cv2.imwrite(write_file_path,rotated)

[INFO] angle: -0.000


True

In [58]:
from PIL import Image
file_loc = write_file_path
image_file = Image.open(file_loc)
image_file = image_file.convert('1')
image_file.save(file_loc)

In [68]:
with io.open(file_loc, 'rb') as image_file:
    content = image_file.read()

In [69]:
client = vision.ImageAnnotatorClient()
image = vision.types.Image(content=content)
response = client.text_detection(image=image)

In [70]:
DictResponse=MessageToDict(response)
WordsAndCoordinates=DictResponse['textAnnotations'][1:]

In [71]:
word_list=[]
llx_list=[]
lly_list=[]
lrx_list=[]
lry_list=[]
urx_list=[]
ury_list=[]
ulx_list=[]
uly_list=[]
for i in range(0,len(WordsAndCoordinates)):
    word_list.append(WordsAndCoordinates[i]['description'])
    llx_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][0]['x'])
    lly_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][0]['y'])
    lrx_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][1]['x'])
    lry_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][1]['y'])
    urx_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][2]['x'])
    ury_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][2]['y'])
    ulx_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][3]['x'])
    uly_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][3]['y'])

In [72]:
WordsAndCoordinatesDict={"Word":word_list,'llx':llx_list,'lly':lly_list,
                         'lrx':lrx_list,'lry':lry_list,'urx':urx_list,
                         'ury':ury_list,'ulx':ulx_list,'uly':uly_list}

In [73]:
import pandas as pd
WordsAndCoordinatesDF = pd.DataFrame.from_dict(WordsAndCoordinatesDict)

In [74]:
WordsAndCoordinatesDF.head(20)

,Word,llx,lly,lrx,lry,urx,ury,ulx,uly
0,Western,297,42,369,46,368,60,296,56
1,Australia,377,47,452,52,451,65,376,61
2,Registration,513,54,573,54,573,66,513,66
3,Number,579,57,619,57,619,66,579,66
4,0007011/2019,525,69,607,71,607,80,525,78
5,Certificate,524,82,569,83,569,95,524,94
6,Number,575,83,617,84,617,96,575,95
7,00030500354,531,98,602,100,602,110,531,108
8,DEATH,282,67,343,71,342,84,281,80
9,CERTIFICATE,351,71,469,78,468,92,350,85


In [75]:
lly_ll=WordsAndCoordinatesDF[WordsAndCoordinatesDF['Word'].isin(["Surname"])].head(1)['lly'].values[0]-40

In [76]:
lly_ul=WordsAndCoordinatesDF[WordsAndCoordinatesDF['Word'].isin(["birth"])]['lly'].values[0]+40

In [77]:
WordsAndCoordinatesDFCopy=WordsAndCoordinatesDF[WordsAndCoordinatesDF['lly'].between(lly_ll,lly_ul)]
WordsAndCoordinatesDFCopy=WordsAndCoordinatesDFCopy.sort_values(by=['lly'])

In [78]:
WordsAndCoordinatesDFCopy

,Word,llx,lly,lrx,lry,urx,ury,ulx,uly
5,Certificate,524,82,569,83,569,95,524,94
6,Number,575,83,617,84,617,96,575,95
10,Brths,292,85,313,86,313,92,292,91
11,Deaths,318,87,344,89,344,95,318,93
12,and,347,89,360,90,360,96,347,95
...,...,...,...,...,...,...,...,...,...
73,Father's,97,232,136,233,136,241,97,240
74,Surname,140,233,184,234,184,243,140,242
69,Knowles,273,237,313,238,313,249,273,248
75,Maiden,136,247,171,248,171,257,136,256


In [79]:
lly_surname=WordsAndCoordinatesDFCopy[WordsAndCoordinatesDFCopy['Word'].isin(['Surname'])]['lly'].values[0]
lly_ll=lly_surname-20
lly_ul=lly_surname+20
SurnameList=list(WordsAndCoordinatesDFCopy[(WordsAndCoordinatesDFCopy['Word']!='Surname') & (WordsAndCoordinatesDFCopy['lly'].between(lly_ll,lly_ul))][['Word']].sort_index()['Word'])
SurnameList
Surname=" "
Surname=Surname.join(SurnameList)
Surname

'00030500354 Registration Act DECEASED Given Names Knowles Barry'

In [80]:
lly_Given=WordsAndCoordinatesDFCopy[WordsAndCoordinatesDFCopy['Word'].isin(['Given'])]['lly'].values[0]
lly_ll=lly_Given-20
lly_ul=lly_Given+20
GivenNameList=list(WordsAndCoordinatesDFCopy[(~WordsAndCoordinatesDFCopy['Word'].isin(['Given','Names'])) & (WordsAndCoordinatesDFCopy['lly'].between(lly_ll,lly_ul))][['Word']].sort_index()['Word'])
GivenName=" "
GivenName=GivenName.join(GivenNameList)
GivenName

'Surname Place of death Knowles Barry'

In [81]:
lly_place_of_death=WordsAndCoordinatesDFCopy[WordsAndCoordinatesDFCopy['Word'].isin(['Place'])].head(1)['lly'].values[0]
lly_ll=lly_place_of_death-20
lly_ul=lly_place_of_death+20
PlaceOfDeathList=list(WordsAndCoordinatesDFCopy[(~WordsAndCoordinatesDFCopy['Word'].isin(['Place','of','death'])) & (WordsAndCoordinatesDFCopy['lly'].between(lly_ll,lly_ul))][['Word']].sort_index()['Word'])
PlaceOfDeath=" "
PlaceOfDeath=PlaceOfDeath.join(PlaceOfDeathList)
PlaceOfDeath

'Given Names Knowles Barry Fiona Staniey Hospital, 102-118 Murdoch Drive, Murdoch, Western'

In [82]:
lly_date_of_death=WordsAndCoordinatesDFCopy[WordsAndCoordinatesDFCopy['Word'].isin(['Date'])].head(1)['lly'].values[0]
lly_ll=lly_date_of_death-20
lly_ul=lly_date_of_death+20
DateOfDeathList=list(WordsAndCoordinatesDFCopy[(~WordsAndCoordinatesDFCopy['Word'].isin(['Date','of','death'])) & (WordsAndCoordinatesDFCopy['lly'].between(lly_ll,lly_ul))][['Word']].sort_index()['Word'])
DateOfDeath=" "
DateOfDeath=DateOfDeath.join(DateOfDeathList)
DateOfDeath

'Fiona Staniey Hospital, 102-118 Murdoch Drive, Murdoch, Western Australia Occupation & Sex 28 May 2019'

In [31]:
lly_occupation_and_sex=WordsAndCoordinatesDFCopy[WordsAndCoordinatesDFCopy['Word'].isin(['Occupation'])].head(1)['lly'].values[0]
lly_ll=lly_occupation_and_sex-20
lly_ul=lly_occupation_and_sex+20
OccupationAndSexList=list(WordsAndCoordinatesDFCopy[(~WordsAndCoordinatesDFCopy['Word'].isin(['Occupation','&','Sex'])) & (WordsAndCoordinatesDFCopy['lly'].between(lly_ll,lly_ul))][['Word']].sort_index()['Word'])
OccupationAndSex=" "
OccupationAndSex=OccupationAndSex.join(OccupationAndSexList)
OccupationAndSex

'Labourer Male'

In [32]:
lly_age_and_dob=WordsAndCoordinatesDFCopy[WordsAndCoordinatesDFCopy['Word'].isin(['Age'])].head(1)['lly'].values[0]
lly_ll=lly_age_and_dob-20
lly_ul=lly_age_and_dob+20
AgeAndDOBList=list(WordsAndCoordinatesDFCopy[(~WordsAndCoordinatesDFCopy['Word'].isin(['Age','&','Date','of','Birth'])) & (WordsAndCoordinatesDFCopy['lly'].between(lly_ll,lly_ul))][['Word']].sort_index()['Word'])
AgeAndDOB=" "
AgeAndDOB=AgeAndDOB.join(AgeAndDOBList)
AgeAndDOB

'75 years 30 August 1944'

In [38]:
lly_usual_address=WordsAndCoordinatesDFCopy[WordsAndCoordinatesDFCopy['Word'].isin(['address'])].head(1)['lly'].values[0]
lly_ll=lly_usual_address-36
lly_ul=lly_usual_address+36
UsualAddressList=list(WordsAndCoordinatesDFCopy[(~WordsAndCoordinatesDFCopy['Word'].isin(['Usual','address'])) & (WordsAndCoordinatesDFCopy['lly'].between(lly_ll,lly_ul))][['Word']].sort_index()['Word'])
UsualAddress=" "
UsualAddress=UsualAddress.join(UsualAddressList)
UsualAddress

'Belvedere Caravan Park, 76/153 Mandurah Terrace, Mandurah, Western Australia, Australia'

In [39]:
lly_place_of_birth=WordsAndCoordinatesDFCopy[WordsAndCoordinatesDFCopy['Word'].isin(['birth'])].head(1)['lly'].values[0]
lly_ll=lly_place_of_birth-20
lly_ul=lly_place_of_birth+20
PlaceOfBirthList=list(WordsAndCoordinatesDFCopy[(~WordsAndCoordinatesDFCopy['Word'].isin(['Place','of','birth'])) & (WordsAndCoordinatesDFCopy['lly'].between(lly_ll,lly_ul))][['Word']].sort_index()['Word'])
PlaceOfBirth=" "
PlaceOfBirth=PlaceOfBirth.join(PlaceOfBirthList)
PlaceOfBirth

'Government Hospital, Kalgoorlie, Western Australia, Australia'